In [1]:
import pandas as pd
import numpy as np
from lxml import etree
import xml.etree.ElementTree as ET
import time
import math
import pylab as pl
import matplotlib.pyplot as plt
import seaborn as sns # improves plot aesthetics

# GER EST

In [77]:
xml_positions = etree.parse('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##arbeitsplatop

In [78]:
kpimerged = pd.read_excel('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') #arbeit

In [79]:
path='C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/'    #arbeit

In [ ]:
xml_positions = etree.parse('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##privat
kpimerged = pd.read_excel('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') ##privat

In [80]:
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)

In [6]:
class DataReader:
  """
  Main class to read in xml files provided by the DFB.

  Returns:
      ElementTree root: The root of the xml document.

  """

  def __init__(self,xml_file):
    self.xml_root = self._load_data(xml_file)


  def _load_data(self,xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root

  def create_dataframe(self,df_cols=None,findall_string=".//Object",time_search=None):
    """
    Create a dataframe from the xml inputs

    Args:
        df_cols (list): Optional, the column names. If no names given, columns
          detected automatically
        findall_string (str): The xpath string to iterate trees (https://www.w3schools.com/xml/xml_xpath.asp)
        time_search (float): Will currently look at parent if it's an Event, it
          will compare the time_search to the EventTime. I.e. this is a means of getting
          all passes before the frame time.

    Returns:
        pd.dataframe
    """

    populate_column_names=False
    if df_cols is None:
        df_cols = []
        populate_column_names=True

    out_df = pd.DataFrame(columns = df_cols)

    parent_map = dict((c, p) for p in self.xml_root.getiterator() for c in p)
    for node in self.xml_root.findall(findall_string):
        if time_search is not None:
          if (parent_map[node].tag != "Event"): continue
          event_time = parent_map[node].attrib["EventTime"]
          ts = ciso8601.parse_datetime(event_time)
          # to get time in seconds:
          seconds=time.mktime(ts.timetuple())
          # If timestamp is larger than the search, exit
          if seconds > time_search: continue

        # If column names are not provided, get them
        if populate_column_names==True:
            for key,value in node.attrib.items():
                df_cols.append(key)
                populate_column_names=False

        res = []
        for c in df_cols:
            res.append(node.attrib.get(c))
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index=True)
    return out_df


In [7]:
def get_tracking_data(xml_positions, half='firstHalf'):
        """Returns a dataframe with the tracking (position: x, y, z) of all players + the ball
        
        Dropping rows where ball positions is NAN

        +-------+-----------+-----------+
        |       |   BALL    |  TeamId   |
        +       +-----------+-----------+
        |       |  BallId   | PlayerId  |
        + N | T +---+---+---+---+---+---+
        |       | X | Y | Z | X | Y | A |
        +=======+===+===+===+===+===+===+
        | 0 | 0 |123|456|789|012|456|789|
        +-------+-----------+-----------+

        :param xml_positions: the parsed xml tracking data (lxml object)
        :param half: the description of the half: either "firstHalf" or "secondHalf", etc.
        :return: a dataframe containing the tracking data: position of all players and the ball
        """
        def change_type(df, frameset):
            """
            change type of columns
            different columns names and tpyes for the ball and for the rest of the players
            """
            if frameset.attrib.get('TeamId') == 'BALL':
                return df.astype(
                    # change types of columns for the ball
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        'Z':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                        'BallPossession':int, 
                        'BallStatus':int,
                    }, 
                    errors='raise'
                )
            else:
                return df.astype(
                    # change types of columns for the players
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                    }, 
                    errors='raise'
                )
        

        def create_columns(df, frameset):
            """A utility function to create a multiindex columns"""
            person_id = frameset.attrib.get('PersonId')
            team_id = frameset.attrib.get('TeamId')
            df.columns = pd.MultiIndex.from_product(
                [[team_id], [person_id], df.columns],
                names=['TeamId', 'PersonId', 'Position']
            )
            return df

        tracking_players = pd.concat([
            pd.DataFrame.from_records(
                [dict(frame.attrib) for frame in frameset.getchildren()]
            ).pipe(change_type, frameset=frameset)
             .set_index(['N', 'T'])
             .pipe(create_columns, frameset=frameset)
            for frameset in xml_positions.xpath(F'//Positions/FrameSet[@GameSection = "{half}"]')
        ], axis=1, sort=False)
        
        # this is casting the 'T' values in the index to datetime with the correct time zone
        tracking_players.index.set_levels(pd.to_datetime(tracking_players.index.get_level_values(1)), level=1, inplace = True)

        return tracking_players

In [81]:
positions=get_tracking_data(xml_positions)     
##create dataframe from xml input
positions2=get_tracking_data(xml_positions,"secondHalf")

In [9]:
positions2=get_tracking_data(xml_positions,"secondHalf")

In [82]:
#filter all the passes from the synchronized eventdata
passdf=kpimergednew[(kpimergednew['SUBTYPE']=='Pass')]  
passdf.reset_index(drop=True)
#define variables of the two teams
CUID1=passdf.CUID.unique()[0]
CUID2=passdf.CUID.unique()[1]

In [11]:
##get the player ids of the team with club id 1
cuid1_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID1)
cuid1_team_data = DataReader(path+cuid1_player_info)
cuid1_team_meta_df = cuid1_team_data.create_dataframe()
cuid1_player_IDs = cuid1_team_meta_df.ObjectId.tolist()
##get the player ids of the team with club id 2
cuid2_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID2)
cuid2_team_data = DataReader(path+cuid2_player_info)
cuid2_team_meta_df = cuid2_team_data.create_dataframe()
cuid2_player_IDs = cuid2_team_meta_df.ObjectId.tolist()

In [83]:
matchinfo = {
    CUID1: {
        "BallPoss":1,
        "TeamID":CUID1,
    },
    CUID2: {
        "BallPoss":2,
        "TeamID":CUID2,
    },
    1: {
        "Dataframe":positions,
    },
    2: {
        "Dataframe":positions2,
    }
}

In [13]:
def getdirectionofplay(teamid,half):
    if matchinfo.get(half)["Dataframe"].loc[matchinfo.get(half)["Dataframe"].index.get_level_values(0)[0],(teamid,slice(None),"X")].mean(axis=1).iat[0] < 0:
        dirofplay = 'left to right'
    if matchinfo.get(half)["Dataframe"].loc[matchinfo.get(half)["Dataframe"].index.get_level_values(0)[0],(teamid,slice(None),"X")].mean(axis=1).iat[0] > 0:
        dirofplay = 'right to left'
    return  dirofplay

In [14]:
def getteamsheet(teamid,half):
    teamsheet=matchinfo.get(half)["Dataframe"].loc[matchinfo.get(half)["Dataframe"].index.get_level_values(0)[0],(teamid,slice(None),"X")].columns.get_level_values(level=1).unique()
    return teamsheet

In [84]:
matchinfo = {
    CUID1: {
        1:getdirectionofplay(CUID1,1),
        2:getdirectionofplay(CUID1,2),
        "BallPoss":1,
        "TeamID":CUID1,
    },
    CUID2: {
        1:getdirectionofplay(CUID2,1),
        2:getdirectionofplay(CUID2,2),
        "BallPoss":2,
        "TeamID":CUID2,
    },
    1: {
        "Dataframe":positions,
        CUID1:getteamsheet(CUID1,1),
        CUID2:getteamsheet(CUID2,1),
        "Startframe":positions.index.get_level_values(0)[0],
        "Endframe":positions.index.get_level_values(0)[positions.shape[0]-1],
    },
    2: {
        "Dataframe":positions2,
        CUID1:getteamsheet(CUID1,2),
        CUID2:getteamsheet(CUID2,2),
        "Startframe":positions2.index.get_level_values(0)[0],
        "Endframe":positions2.index.get_level_values(0)[positions.shape[0]-1],
    },
    "left to right":{
        "Goalline":52.5
    },
    "right to left":{
        "Goalline":-52.5
    }
}

In [75]:
positions.shape[0]

67675

In [81]:
positions.index.get_level_values(0)[positions.shape[0]-1]

77674

In [85]:
## initialize a dictionary with the kickoff times for the first and second half
half={} # frames der kickoff zeiten in frames
kickoffs=kpimergednew[(kpimergednew['SUBTYPE']=='Kickoff')].index.tolist()
for i in kickoffs:
    if i != 0:
        framedif=kpimergednew.iloc[i].N -kpimergednew.iloc[i-2].N
        if framedif > 10000:
            half['2nd half']=kpimergednew.iloc[i].N
    else:
        half['1st half']=kpimergednew.iloc[i].N

In [86]:
##directionplayframe gives the direction of play of a given team at a specific frame
def directionplayframe(frame,teamid):
    ###Check for teamid
    if teamid == CUID1:
        gk_df=cuid1_team_meta_df.loc[cuid1_team_meta_df['PlayingPositionEnglish']=='goalkeeper'] #select goalkeeper information from team with clubid1
        gks=gk_df['ObjectId'] #select only the ids
        frame1=half['1st half'] #define start of first half
        frame2=half['2nd half'] #define start of second half
        if frame<frame2:  #if the selected frame is smaller than frame2 the frame lies within the first half 
            for items in gks.iteritems():  # iterating through the list of goalkeeper
                if (teamid,items[1],'X') in positions.columns:  #if the id is part of the tracking data column names then continue
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]  ##create a df with the x and y coordinates of the playing goalkeeper
                    if gkpos[teamid,items[1],'X'].mean() < 0:   #if the mean of the x coordinate is smaller than 0 the direction of play is from left to right
                        dirofplay = 'left to right'
                        return dirofplay
                    else:
                        dirofplay = 'right to left'
                        return dirofplay
        else:
            for items in gks.iteritems():
                if (teamid,items[1],'X') in positions.columns:
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'right to left'
                        return dirofplay
                    else:
                        dirofplay = 'left to right'
                        return dirofplay
    if teamid == CUID2:
        gk_df=cuid2_team_meta_df.loc[cuid2_team_meta_df['PlayingPositionEnglish']=='goalkeeper']
        gks=gk_df['ObjectId']
        frame1=half['1st half']
        frame2=half['2nd half']
        if frame<frame2:
            for items in gks.iteritems():
                if (teamid,items[1],'X') in positions.columns:
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'left to right'
                        return dirofplay
                    else:
                        dirofplay = 'right to left'
                        return dirofplay
        else:
            for items in gks.iteritems():
                if (teamid,items[1],'X') in positions.columns:
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'right to left'
                        return dirofplay
                    else:
                        dirofplay = 'left to right'
                        return dirofplay

In [87]:
##function that returns 1 if the pass is in the direction of play 90 degrees and 0 if not
def qpass90 (row):
    #PUID1=row.PUID1   
    #PUID2=row.PUID2
    t1=int(row.N)
    #t2=int(row.RECFRM)
    CUID=row.CUID
    p1=np.array([float(row.X_TRACKING),float(row.Y_TRACKING)])
    p2=np.array([float(row.XRec),float(row.YRec)])
    if directionplayframe(t1,CUID) == 'left to right':
        triangle=np.array([float(52.5),float(row.Y_TRACKING)])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)
    if directionplayframe(t1,CUID) == 'right to left':
        triangle=np.array([float(-52.5),float(row.Y_TRACKING)])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)

In [117]:
#new
##function that returns 1 if the pass is in the direction of play 90 degrees and 0 if not
def qpass90 (row):
    t1=int(row.N)
    CUID=row.CUID
    p1=np.array([float(row.X_TRACKING),float(row.Y_TRACKING)])
    p2=np.array([float(row.XRec),float(row.YRec)])
    if (t1>matchinfo.get(1)["Startframe"]) and (t1<matchinfo.get(1)["Endframe"]):
        half=1
        triangle=np.array([float(matchinfo.get(matchinfo.get(CUID)[half])["Goalline"]),float(row.Y_TRACKING)])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)
    if (t1>matchinfo.get(2)["Startframe"]) and (t1<matchinfo.get(2)["Endframe"]):
        half=2
        triangle=np.array([float(matchinfo.get(matchinfo.get(CUID)[half])["Goalline"]),float(row.Y_TRACKING)])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)

In [110]:
###old
def qpass90quote (df,eval):
    result=[]
    for i in range(df.shape[0]):
        if i==(df.shape[0]-1):
            result.append(0)
        else:
            t1=int(df.loc[df.index[i],'N'])
            CUID=df.loc[df.index[i],'CUID']
            p1=np.array([float(df.loc[df.index[i],'X_TRACKING']),float(df.loc[df.index[i],'Y_TRACKING'])])
            p2=np.array([float(kpimergednew.loc[df.index[i]+1,'X_TRACKING']),float(kpimergednew.loc[df.index[i]+1,'Y_TRACKING'])])
            if directionplayframe(t1,CUID) == 'left to right':
                triangle=np.array([float(52.5),float(df.loc[df.index[i],'Y_TRACKING'])])
                passline=p2-p1
                passtor=triangle-p1
                cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
                angle = np.arccos(cosine_angle)
                if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                    result.append(1)
                else:
                    result.append(0)
            if directionplayframe(t1,CUID) == 'right to left':
                triangle=np.array([float(-52.5),float(df.loc[df.index[i],'Y_TRACKING'])])
                passline=p2-p1
                passtor=triangle-p1
                cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
                angle = np.arccos(cosine_angle)
                if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                    result.append(1)
                else:
                    result.append(0)
    if eval == 'successful':
        return(result.count(1))
    if eval == 'unsuccessful':
        return(result.count(0))

In [135]:
#new
def qpass90quote (df,eval):
    result=[]
    for i in range(df.shape[0]):
        if i==(df.shape[0]-1):
            result.append(0)
        else:
            t1=int(df.loc[df.index[i],'N'])
            CUID=df.loc[df.index[i],'CUID']
            p1=np.array([float(df.loc[df.index[i],'X_TRACKING']),float(df.loc[df.index[i],'Y_TRACKING'])])
            p2=np.array([float(kpimergednew.loc[df.index[i]+1,'X_TRACKING']),float(kpimergednew.loc[df.index[i]+1,'Y_TRACKING'])])
            if (t1>matchinfo.get(1)["Startframe"]) and (t1<matchinfo.get(1)["Endframe"]):
                half=1
                triangle=np.array([float(matchinfo.get(matchinfo.get(CUID)[half])["Goalline"]),float(df.loc[df.index[i],'Y_TRACKING'])])
                passline=p2-p1
                passtor=triangle-p1
                cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
                angle = np.arccos(cosine_angle)
                if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                    result.append(1)
                else:
                    result.append(0)
            if (t1>matchinfo.get(2)["Startframe"]) and (t1<matchinfo.get(2)["Endframe"]):
                half=2
                triangle=np.array([float(matchinfo.get(matchinfo.get(CUID)[half])["Goalline"]),float(df.loc[df.index[i],'Y_TRACKING'])])
                passline=p2-p1
                passtor=triangle-p1
                cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
                angle = np.arccos(cosine_angle)
                if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                    result.append(1)
                else:
                    result.append(0)
    if eval == 'successful':
        return(result.count(1))
    if eval == 'unsuccessful':
        return(result.count(0))

In [89]:
##old
def qpass90goal (row):
    t1=int(row.N)
    CUID=row.CUID
    p1=np.array([float(row.X_TRACKING),float(row.Y_TRACKING)])
    p2=np.array([float(row.XRec),float(row.YRec)])
    if directionplayframe(t1,CUID) == 'left to right':
        triangle=np.array([float(52.5),0])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)
    if directionplayframe(t1,CUID) == 'right to left':
        triangle=np.array([float(-52.5),0])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)

In [115]:
##new
def qpass90goal (row):
    #PUID1=row.PUID1
    #PUID2=row.PUID2
    t1=int(row.N)
    #t2=int(row.RECFRM)
    CUID=row.CUID
    p1=np.array([float(row.X_TRACKING),float(row.Y_TRACKING)])
    p2=np.array([float(row.XRec),float(row.YRec)])
    if (t1>matchinfo.get(1)["Startframe"]) and (t1<matchinfo.get(1)["Endframe"]):
        half=1
        triangle=np.array([float(matchinfo.get(matchinfo.get(CUID)[half])["Goalline"]),0])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)
    if (t1>matchinfo.get(2)["Startframe"]) and (t1<matchinfo.get(2)["Endframe"]):
        half=2
        triangle=np.array([float(matchinfo.get(matchinfo.get(CUID)[half])["Goalline"]),0])
        passline=p2-p1
        passtor=triangle-p1
        cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
        angle = np.arccos(cosine_angle)
        if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
            return(1)
        else:
            return(0)

In [90]:
##old
def qpass90goalquote (df,eval):
    result=[]
    for i in range(df.shape[0]):
        t1=int(df.loc[df.index[i],'N'])
        CUID=df.loc[df.index[i],'CUID']
        p1=np.array([float(df.loc[df.index[i],'X_TRACKING']),float(df.loc[df.index[i],'Y_TRACKING'])])
        p2=np.array([float(kpimergednew.loc[df.index[i]+1,'X_TRACKING']),float(kpimergednew.loc[df.index[i]+1,'Y_TRACKING'])])
        if directionplayframe(t1,CUID) == 'left to right':
            triangle=np.array([float(52.5),0])
            passline=p2-p1
            passtor=triangle-p1
            cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
            angle = np.arccos(cosine_angle)
            if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                result.append(1)
            else:
                result.append(0)
        if directionplayframe(t1,CUID) == 'right to left':
            triangle=np.array([float(-52.5),0])
            passline=p2-p1
            passtor=triangle-p1
            cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
            angle = np.arccos(cosine_angle)
            if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                result.append(1)
            else:
                result.append(0)
    if eval == 'successful':
        return(result.count(1))
    if eval == 'unsuccessful':
        return(result.count(0))

In [114]:
##bestimmt die anzahl der pässe in spielrichtung 90, die nicht erfolgreich waren.
#new
def qpass90goalquote (df,eval):
    result=[]
    for i in range(df.shape[0]):
        if i==(df.shape[0]-1):
            result.append(0)
        else:
            t1=int(df.loc[df.index[i],'N'])
            CUID=df.loc[df.index[i],'CUID']
            p1=np.array([float(df.loc[df.index[i],'X_TRACKING']),float(df.loc[df.index[i],'Y_TRACKING'])])
            p2=np.array([float(kpimergednew.loc[df.index[i]+1,'X_TRACKING']),float(kpimergednew.loc[df.index[i]+1,'Y_TRACKING'])])
            if (t1>matchinfo.get(1)["Startframe"]) and (t1<matchinfo.get(1)["Endframe"]):
                half=1
                triangle=np.array([float(matchinfo.get(matchinfo.get(CUID)[half])["Goalline"]),0])
                passline=p2-p1
                passtor=triangle-p1
                cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
                angle = np.arccos(cosine_angle)
                if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                    result.append(1)
                else:
                    result.append(0)
            if (t1>matchinfo.get(2)["Startframe"]) and (t1<matchinfo.get(2)["Endframe"]):
                half=2
                triangle=np.array([float(matchinfo.get(matchinfo.get(CUID)[half])["Goalline"]),0])
                passline=p2-p1
                passtor=triangle-p1
                cosine_angle = np.dot(passline, passtor) / (np.linalg.norm(passline) * np.linalg.norm(passtor))
                angle = np.arccos(cosine_angle)
                if np.degrees(angle) <= 45 or np.degrees(angle) >= 315:
                    result.append(1)
                else:
                    result.append(0)
    if eval == 'successful':
        return(result.count(1))
    if eval == 'unsuccessful':
        return(result.count(0))

In [118]:
passdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID1)]
passsuccessdf1=passdf1[(passdf1['EVALUATION']=='successfullyComplete') | (passdf1['EVALUATION']=='successful')]
passsuccessrecdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID1)& (kpimergednew.NORECEIVER==False)]
passunsuccessdf1=passdf1[passdf1['EVALUATION']=='unsuccessful']
passdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID2)]
passsuccessdf2=passdf2[(passdf2['EVALUATION']=='successfullyComplete') | (passdf2['EVALUATION']=='successful')]
passsuccessrecdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID2) & (kpimergednew.NORECEIVER==False)]
passunsuccessdf2=passdf2[passdf2['EVALUATION']=='unsuccessful']

In [119]:
#apply the two functions on the dataframe
# runtime around 3 mins !
passsuccessrecdf1['qpass90']=passsuccessrecdf1.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf2['qpass90']=passsuccessrecdf2.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf1['qpass90goal']=passsuccessrecdf1.apply(lambda row:qpass90goal(row),axis=1)
passsuccessrecdf2['qpass90goal']=passsuccessrecdf2.apply(lambda row:qpass90goal(row),axis=1)

C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:

In [120]:
cuid190ges=passsuccessrecdf1['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf1,'successful')
cuid290ges=passsuccessrecdf2['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf2,'successful')
cuid190goalges=passsuccessrecdf1['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf1,'successful')
cuid290goalges=passsuccessrecdf2['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf2,'successful')

C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


In [121]:
#Total: Anzahl aller Pässe
#Successful: erfolgreiche Pässe
#Unsuccessful: fehlgeschlagene Pässe
#Successful in Spielrichtung: Erfolgreiche Pässe in Spielrichtung
#Successful gegen Spielrichtung: Alle Pässe, die nicht in Spielrichtung 90 Grad gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung: Alle Pässe, die in Spielrichtung 90 Grad gespielt wurden 
#Successful in Spielrichtung (Tormitte): Erfolgreiche Pässe in Spielrichtung 90 Grad zur Tormitte
#Successful gegen Spielrichtung (Tormitte): Alle Pässe, die nicht in Spielrichtung 90 Grad zur Tormitte gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung (Tormitte): Alle Pässe, die in Spielrichtung 90 Grad zur Tormitte gespielt wurden 
passstatistic = {CUID1: [passdf1.shape[0],passsuccessrecdf1.shape[0],passunsuccessdf1.shape[0],passsuccessrecdf1['qpass90'].value_counts()[1],passsuccessrecdf1['qpass90'].value_counts()[0],cuid190ges,passsuccessrecdf1['qpass90goal'].value_counts()[1],passsuccessrecdf1['qpass90goal'].value_counts()[0],cuid190goalges],
                 CUID2: [passdf2.shape[0],passsuccessrecdf2.shape[0],passunsuccessdf2.shape[0],passsuccessrecdf2['qpass90'].value_counts()[1],passsuccessrecdf2['qpass90'].value_counts()[0],cuid290ges,passsuccessrecdf2['qpass90goal'].value_counts()[1],passsuccessrecdf2['qpass90goal'].value_counts()[0],cuid290goalges]}
passstatisticdf = pd.DataFrame(passstatistic,columns= [CUID1, CUID2],
                 index=['Total','Successful','Unsuccessful','Successful in Spielrichtung','Successful gegen Spielrichtung','Total Pässe in Spielrichtung','Successful in Spielrichtung (Tormitte)','Successful gegen Spielrichtung (Tormitte)','Total in Spielrichtung (Tormitte)'])
print(passstatisticdf)

                                           DFL-CLU-000N8Y  DFL-CLU-000N8Z
Total                                                1020             230
Successful                                            942             165
Unsuccessful                                           61              65
Successful in Spielrichtung                           204              46
Successful gegen Spielrichtung                        716             114
Total Pässe in Spielrichtung                          234              98
Successful in Spielrichtung (Tormitte)                217              47
Successful gegen Spielrichtung (Tormitte)             703             113
Total in Spielrichtung (Tormitte)                     253             100


In [122]:
passquotetotal1=passsuccessrecdf1.shape[0]/passdf1.shape[0]
passquotetotal2=passsuccessrecdf2.shape[0]/passdf2.shape[0]
passquote901=passsuccessrecdf1['qpass90'].value_counts()[1]/cuid190ges
passquote902=passsuccessrecdf2['qpass90'].value_counts()[1]/cuid290ges
passquote90goal1=passsuccessrecdf1['qpass90goal'].value_counts()[1]/cuid190goalges
passquote90goal2=passsuccessrecdf2['qpass90goal'].value_counts()[1]/cuid290goalges

In [123]:
#Passquote Total: Erfolgreiche Pässe
#Passquote Total in Spielrichtung: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung gespielt wurden
#Passquote Total in Spielrichtung Tormitte: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung Tormitte gespielt wurden
passstatisticproz = {CUID1: [passquotetotal1,passquote901,passquote90goal1],
                 CUID2: [passquotetotal2,passquote902,passquote90goal2]}
passstatisticprozdf = pd.DataFrame(passstatisticproz,columns= [CUID1, CUID2],
                 index=['Passquote Total','Passquote Total in Spielrichtung','Passquote Total in Spielrichtung Tormitte'])
print(passstatisticprozdf)

                                           DFL-CLU-000N8Y  DFL-CLU-000N8Z
Passquote Total                                  0.923529        0.717391
Passquote Total in Spielrichtung                 0.871795        0.469388
Passquote Total in Spielrichtung Tormitte        0.857708        0.470000


# GER NED

In [124]:
kpimerged = pd.read_excel('C:/Users/Jung/Desktop/Q-Index Master/data/GER-NED/KPI_MERGED.xlsx') #arbeit

In [125]:
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)

In [126]:
path='C:/Users/Jung/Desktop/Q-Index Master/data/GER-NED/'    #arbeit

In [127]:
xml_positions = etree.parse('C:/Users/Jung/Desktop/Q-Index Master/data/GER-NED/DFL_04_02_positions_raw_DFL-COM-000001_DFL-MAT-X03BWS.xml') ##arbeitsplatop
positions=get_tracking_data(xml_positions)     
##create dataframe from xml input
positions2=get_tracking_data(xml_positions,"secondHalf")  

In [128]:
#filter all the passes from the synchronized eventdata
passdf=kpimergednew[(kpimergednew['SUBTYPE']=='Pass')]  
passdf.reset_index(drop=True)


,EVENT_ID,MUID,MATCH_DAY,PUID1,PUID2,SHIRT_NUMBER1,SHIRT_NUMBER2,NOMATCH,GDCP_EVENT_TIME,TRACKING_TIME,...,AssistTyp,DISTANCE,ANGLE,FREEKICK,FOOT,SPEED,PRESSURE_ON_SHOT,DEFENDERS,DISTANCE_GOALKEEPER,GK_INGOAL
0,485823,DFL-MAT-X03BWS,1,DFL-OBJ-0027G6,DFL-OBJ-0000I6,20,8,False,2019-09-06 20:45:46,2019-09-06 20:45:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,485832,DFL-MAT-X03BWS,1,DFL-OBJ-0000I4,DFL-OBJ-00021U,1,9,False,2019-09-06 20:45:56,2019-09-06 20:45:53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,485826,DFL-MAT-X03BWS,1,DFL-OBJ-002G75,DFL-OBJ-002G70,8,4,False,2019-09-06 20:46:05,2019-09-06 20:45:56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,485831,DFL-MAT-X03BWS,1,DFL-OBJ-0001UP,DFL-OBJ-0000I4,5,1,True,2019-09-06 20:45:53,2019-09-06 20:46:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,485795,DFL-MAT-X03BWS,1,DFL-OBJ-002G6N,DFL-OBJ-002G75,21,8,False,2019-09-06 20:46:03,2019-09-06 20:46:01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,485828,DFL-MAT-X03BWS,1,DFL-OBJ-0000ZS,DFL-OBJ-002G6P,4,17,True,2019-09-06 20:46:01,2019-09-06 20:46:04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,485825,DFL-MAT-X03BWS,1,DFL-OBJ-002G6P,DFL-OBJ-002G6N,17,21,True,2019-09-06 20:46:02,2019-09-06 20:46:05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,485813,DFL-MAT-X03BWS,1,DFL-OBJ-002G6O,DFL-OBJ-002G6R,3,15,False,2019-09-06 20:46:12,2019-09-06 20:46:08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,485824,DFL-MAT-X03BWS,1,DFL-OBJ-0000I6,DFL-OBJ-0001UP,8,5,False,2019-09-06 20:45:51,2019-09-06 20:46:12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,485810,DFL-MAT-X03BWS,1,DFL-OBJ-002G6R,DFL-OBJ-0000W3,15,9,True,2019-09-06 20:46:15,2019-09-06 20:46:13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
##get the player ids of the team with club id 1
CUID1 = 'DFL-CLU-000N9A'  #Niederlande
#cuid1_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K4_player.xml".format(CUID1)
#cuid1_team_data = DataReader(path+cuid1_player_info)
#cuid1_team_meta_df = cuid1_team_data.create_dataframe()
#cuid1_player_IDs = cuid1_team_meta_df.ObjectId.tolist()
##get the player ids of the team with club id 2
CUID2 = 'DFL-CLU-000N99'  #Deutschland
#cuid2_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K4_player.xml".format(CUID2)
#cuid2_team_data = DataReader(path+cuid2_player_info)
#cuid2_team_meta_df = cuid2_team_data.create_dataframe()
#cuid2_player_IDs = cuid2_team_meta_df.ObjectId.tolist()

In [130]:
matchinfo = {
    CUID1: {
        "BallPoss":1,
        "TeamID":CUID1,
    },
    CUID2: {
        "BallPoss":2,
        "TeamID":CUID2,
    },
    1: {
        "Dataframe":positions,
    },
    2: {
        "Dataframe":positions2,
    }
}

In [14]:
## initialize a dictionary with the kickoff times for the first and second half
half={} # frames der kickoff zeiten in frames
kickoffs=kpimergednew[(kpimergednew['SUBTYPE']=='Kickoff')].index.tolist()
for i in kickoffs:
    if i != 0:
        framedif=kpimergednew.iloc[i].N -kpimergednew.iloc[i-2].N
        if framedif > 10000:
            half['2nd half']=kpimergednew.iloc[i].N
    else:
        half['1st half']=kpimergednew.iloc[i].N

In [131]:
matchinfo = {
    CUID1: {
        1:getdirectionofplay(CUID1,1),
        2:getdirectionofplay(CUID1,2),
        "BallPoss":1,
        "TeamID":CUID1,
    },
    CUID2: {
        1:getdirectionofplay(CUID2,1),
        2:getdirectionofplay(CUID2,2),
        "BallPoss":2,
        "TeamID":CUID2,
    },
    1: {
        "Dataframe":positions,
        CUID1:getteamsheet(CUID1,1),
        CUID2:getteamsheet(CUID2,1),
        "Startframe":positions.index.get_level_values(0)[0],
        "Endframe":positions.index.get_level_values(0)[positions.shape[0]-1],
    },
    2: {
        "Dataframe":positions2,
        CUID1:getteamsheet(CUID1,2),
        CUID2:getteamsheet(CUID2,2),
        "Startframe":positions2.index.get_level_values(0)[0],
        "Endframe":positions2.index.get_level_values(0)[positions.shape[0]-1],
    },
    "left to right":{
        "Goalline":52.5
    },
    "right to left":{
        "Goalline":-52.5
    }
}

In [132]:
passdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID1)]
passsuccessdf1=passdf1[(passdf1['EVALUATION']=='successfullyComplete') | (passdf1['EVALUATION']=='successful')]
passsuccessrecdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID1)& (kpimergednew.NORECEIVER==False)]
passunsuccessdf1=passdf1[passdf1['EVALUATION']=='unsuccessful']
passdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID2)]
passsuccessdf2=passdf2[(passdf2['EVALUATION']=='successfullyComplete') | (passdf2['EVALUATION']=='successful')]
passsuccessrecdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID2) & (kpimergednew.NORECEIVER==False)]
passunsuccessdf2=passdf2[passdf2['EVALUATION']=='unsuccessful']

In [136]:
#apply the two functions on the dataframe
# runtime around 3 mins !
passsuccessrecdf1['qpass90']=passsuccessrecdf1.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf2['qpass90']=passsuccessrecdf2.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf1['qpass90goal']=passsuccessrecdf1.apply(lambda row:qpass90goal(row),axis=1)
passsuccessrecdf2['qpass90goal']=passsuccessrecdf2.apply(lambda row:qpass90goal(row),axis=1)

C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:

In [137]:
cuid190ges=passsuccessrecdf1['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf1,'successful')
cuid290ges=passsuccessrecdf2['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf2,'successful')
cuid190goalges=passsuccessrecdf1['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf1,'successful')
cuid290goalges=passsuccessrecdf2['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf2,'successful')

C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


In [138]:
#Total: Anzahl aller Pässe
#Successful: erfolgreiche Pässe
#Unsuccessful: fehlgeschlagene Pässe
#Successful in Spielrichtung: Erfolgreiche Pässe in Spielrichtung
#Successful gegen Spielrichtung: Alle Pässe, die nicht in Spielrichtung 90 Grad gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung: Alle Pässe, die in Spielrichtung 90 Grad gespielt wurden 
#Successful in Spielrichtung (Tormitte): Erfolgreiche Pässe in Spielrichtung 90 Grad zur Tormitte
#Successful gegen Spielrichtung (Tormitte): Alle Pässe, die nicht in Spielrichtung 90 Grad zur Tormitte gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung (Tormitte): Alle Pässe, die in Spielrichtung 90 Grad zur Tormitte gespielt wurden 
passstatistic = {CUID1: [passdf1.shape[0],passsuccessrecdf1.shape[0],passunsuccessdf1.shape[0],passsuccessrecdf1['qpass90'].value_counts()[1],passsuccessrecdf1['qpass90'].value_counts()[0],cuid190ges,passsuccessrecdf1['qpass90goal'].value_counts()[1],passsuccessrecdf1['qpass90goal'].value_counts()[0],cuid190goalges],
                 CUID2: [passdf2.shape[0],passsuccessrecdf2.shape[0],passunsuccessdf2.shape[0],passsuccessrecdf2['qpass90'].value_counts()[1],passsuccessrecdf2['qpass90'].value_counts()[0],cuid290ges,passsuccessrecdf2['qpass90goal'].value_counts()[1],passsuccessrecdf2['qpass90goal'].value_counts()[0],cuid290goalges]}
passstatisticdf = pd.DataFrame(passstatistic,columns= [CUID1, CUID2],
                 index=['Total','Successful','Unsuccessful','Successful in Spielrichtung','Successful gegen Spielrichtung','Total Pässe in Spielrichtung','Successful in Spielrichtung (Tormitte)','Successful gegen Spielrichtung (Tormitte)','Total in Spielrichtung (Tormitte)'])
print(passstatisticdf)

                                           DFL-CLU-000N9A  DFL-CLU-000N99
Total                                                 571             547
Successful                                            477             470
Unsuccessful                                           65              59
Successful in Spielrichtung                           101             108
Successful gegen Spielrichtung                        363             338
Total Pässe in Spielrichtung                          140             137
Successful in Spielrichtung (Tormitte)                103             112
Successful gegen Spielrichtung (Tormitte)             361             334
Total in Spielrichtung (Tormitte)                     147             144


In [139]:
passquotetotal1=passsuccessrecdf1.shape[0]/passdf1.shape[0]
passquotetotal2=passsuccessrecdf2.shape[0]/passdf2.shape[0]
passquote901=passsuccessrecdf1['qpass90'].value_counts()[1]/cuid190ges
passquote902=passsuccessrecdf2['qpass90'].value_counts()[1]/cuid290ges
passquote90goal1=passsuccessrecdf1['qpass90goal'].value_counts()[1]/cuid190goalges
passquote90goal2=passsuccessrecdf2['qpass90goal'].value_counts()[1]/cuid290goalges

In [140]:
#Passquote Total: Erfolgreiche Pässe
#Passquote Total in Spielrichtung: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung gespielt wurden
#Passquote Total in Spielrichtung Tormitte: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung Tormitte gespielt wurden
passstatisticproz = {CUID1: [passquotetotal1,passquote901,passquote90goal1],
                 CUID2: [passquotetotal2,passquote902,passquote90goal2]}
passstatisticprozdf = pd.DataFrame(passstatisticproz,columns= [CUID1, CUID2],
                 index=['Passquote Total','Passquote Total in Spielrichtung','Passquote Total in Spielrichtung Tormitte'])
print(passstatisticprozdf)

                                           DFL-CLU-000N9A  DFL-CLU-000N99
Passquote Total                                  0.835377        0.859232
Passquote Total in Spielrichtung                 0.721429        0.788321
Passquote Total in Spielrichtung Tormitte        0.700680        0.777778


# GER - BEL U21

In [145]:
path='C:/Users/Jung/Desktop/Q-Index Master/data/GER-BEL/'    #arbeit
kpimerged = pd.read_excel('C:/Users/Jung/Desktop/Q-Index Master/data/GER-BEL/KPI_MERGED.xlsx') #arbeit
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)

xml_positions = etree.parse('C:/Users/Jung/Desktop/Q-Index Master/data/GER-BEL/DFL_04_02_positions_raw_DFL-COM-000001_DFL-MAT-003BWT.xml') ##arbeitsplatop
positions=get_tracking_data(xml_positions)     
positions2=get_tracking_data(xml_positions,"secondHalf")     
##create dataframe from xml input

In [156]:
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)
kpimergednew = kpimergednew.rename({'CUID1': 'CUID'}, axis=1)

In [146]:
#filter all the passes from the synchronized eventdata
passdf=kpimergednew[(kpimergednew['SUBTYPE']=='Pass')]  
passdf.reset_index(drop=True)


,EVENT_ID,MUID,MATCH_DAY,PUID1,Player1,SHIRT_NUMBER1,CUID1,Club1,Club1_Three_Letter_Code,PUID2,...,AssistTyp,DISTANCE,ANGLE,FREEKICK,FOOT,SPEED,PRESSURE_ON_SHOT,DEFENDERS,DISTANCE_GOALKEEPER,GK_INGOAL
0,6594,DFL-MAT-003BWT,1,DFL-OBJ-002G22,Openda Lois,9,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-002G1Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6595,DFL-MAT-003BWT,1,DFL-OBJ-002G1Z,Alexis Saelemaekers,22,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-002G24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6589,DFL-MAT-003BWT,1,DFL-OBJ-002G24,Zinho Vanheusden,3,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-002G1O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6590,DFL-MAT-003BWT,1,DFL-OBJ-002G1O,Mile Svilar,1,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-0028CM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6585,DFL-MAT-003BWT,1,DFL-OBJ-002G1R,Thibault De Smet,5,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-002G1Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6586,DFL-MAT-003BWT,1,DFL-OBJ-002G1Z,Alexis Saelemaekers,22,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-002G1R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6587,DFL-MAT-003BWT,1,DFL-OBJ-002G1R,Thibault De Smet,5,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-0027TM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6588,DFL-MAT-003BWT,1,DFL-OBJ-0027TM,Orel Mangala,8,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-002G21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6608,DFL-MAT-003BWT,1,DFL-OBJ-002G21,Francis Amuzu,11,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-0027TM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6609,DFL-MAT-003BWT,1,DFL-OBJ-0027TM,Orel Mangala,8,DFL-CLU-000N9B,Belgien Nationalmannschaft,NLD,DFL-OBJ-002G21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
##get the player ids of the team with club id 1
CUID1 = 'DFL-CLU-000N9A'  #Deutschland
#cuid1_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K4_player.xml".format(CUID1)
#cuid1_team_data = DataReader(path+cuid1_player_info)
#cuid1_team_meta_df = cuid1_team_data.create_dataframe()
#cuid1_player_IDs = cuid1_team_meta_df.ObjectId.tolist()
##get the player ids of the team with club id 2
CUID2 = 'DFL-CLU-000N9B'  #Belgien
#cuid2_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K4_player.xml".format(CUID2)
#cuid2_team_data = DataReader(path+cuid2_player_info)
#cuid2_team_meta_df = cuid2_team_data.create_dataframe()
#cuid2_player_IDs = cuid2_team_meta_df.ObjectId.tolist()

In [148]:
matchinfo = {
    CUID1: {
        "BallPoss":1,
        "TeamID":CUID1,
    },
    CUID2: {
        "BallPoss":2,
        "TeamID":CUID2,
    },
    1: {
        "Dataframe":positions,
    },
    2: {
        "Dataframe":positions2,
    }
}

In [149]:
matchinfo = {
    CUID1: {
        1:getdirectionofplay(CUID1,1),
        2:getdirectionofplay(CUID1,2),
        "BallPoss":1,
        "TeamID":CUID1,
    },
    CUID2: {
        1:getdirectionofplay(CUID2,1),
        2:getdirectionofplay(CUID2,2),
        "BallPoss":2,
        "TeamID":CUID2,
    },
    1: {
        "Dataframe":positions,
        CUID1:getteamsheet(CUID1,1),
        CUID2:getteamsheet(CUID2,1),
        "Startframe":positions.index.get_level_values(0)[0],
        "Endframe":positions.index.get_level_values(0)[positions.shape[0]-1],
    },
    2: {
        "Dataframe":positions2,
        CUID1:getteamsheet(CUID1,2),
        CUID2:getteamsheet(CUID2,2),
        "Startframe":positions2.index.get_level_values(0)[0],
        "Endframe":positions2.index.get_level_values(0)[positions.shape[0]-1],
    },
    "left to right":{
        "Goalline":52.5
    },
    "right to left":{
        "Goalline":-52.5
    }
}

In [159]:
passdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID1)]
passsuccessdf1=passdf1[(passdf1['EVALUATION']=='successfullyComplete') | (passdf1['EVALUATION']=='successful')]
passsuccessrecdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID1)& (kpimergednew.NORECEIVER==False)]
passunsuccessdf1=passdf1[passdf1['EVALUATION']=='unsuccessful']
passdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID2)]
passsuccessdf2=passdf2[(passdf2['EVALUATION']=='successfullyComplete') | (passdf2['EVALUATION']=='successful')]
passsuccessrecdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID2) & (kpimergednew.NORECEIVER==False)]
passunsuccessdf2=passdf2[passdf2['EVALUATION']=='unsuccessful']

In [160]:
#apply the two functions on the dataframe
# runtime around 3 mins !
passsuccessrecdf1['qpass90']=passsuccessrecdf1.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf2['qpass90']=passsuccessrecdf2.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf1['qpass90goal']=passsuccessrecdf1.apply(lambda row:qpass90goal(row),axis=1)
passsuccessrecdf2['qpass90goal']=passsuccessrecdf2.apply(lambda row:qpass90goal(row),axis=1)

C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [161]:
cuid190ges=passsuccessrecdf1['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf1,'successful')
cuid290ges=passsuccessrecdf2['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf2,'successful')
cuid190goalges=passsuccessrecdf1['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf1,'successful')
cuid290goalges=passsuccessrecdf2['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf2,'successful')

C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


In [162]:
#Total: Anzahl aller Pässe
#Successful: erfolgreiche Pässe
#Unsuccessful: fehlgeschlagene Pässe
#Successful in Spielrichtung: Erfolgreiche Pässe in Spielrichtung
#Successful gegen Spielrichtung: Alle Pässe, die nicht in Spielrichtung 90 Grad gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung: Alle Pässe, die in Spielrichtung 90 Grad gespielt wurden 
#Successful in Spielrichtung (Tormitte): Erfolgreiche Pässe in Spielrichtung 90 Grad zur Tormitte
#Successful gegen Spielrichtung (Tormitte): Alle Pässe, die nicht in Spielrichtung 90 Grad zur Tormitte gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung (Tormitte): Alle Pässe, die in Spielrichtung 90 Grad zur Tormitte gespielt wurden 
passstatistic = {CUID1: [passdf1.shape[0],passsuccessrecdf1.shape[0],passunsuccessdf1.shape[0],passsuccessrecdf1['qpass90'].value_counts()[1],passsuccessrecdf1['qpass90'].value_counts()[0],cuid190ges,passsuccessrecdf1['qpass90goal'].value_counts()[1],passsuccessrecdf1['qpass90goal'].value_counts()[0],cuid190goalges],
                 CUID2: [passdf2.shape[0],passsuccessrecdf2.shape[0],passunsuccessdf2.shape[0],passsuccessrecdf2['qpass90'].value_counts()[1],passsuccessrecdf2['qpass90'].value_counts()[0],cuid290ges,passsuccessrecdf2['qpass90goal'].value_counts()[1],passsuccessrecdf2['qpass90goal'].value_counts()[0],cuid290goalges]}
passstatisticdf = pd.DataFrame(passstatistic,columns= [CUID1, CUID2],
                 index=['Total','Successful','Unsuccessful','Successful in Spielrichtung','Successful gegen Spielrichtung','Total Pässe in Spielrichtung','Successful in Spielrichtung (Tormitte)','Successful gegen Spielrichtung (Tormitte)','Total in Spielrichtung (Tormitte)'])
print(passstatisticdf)

                                           DFL-CLU-000N9A  DFL-CLU-000N9B
Total                                                 521             341
Successful                                            445             269
Unsuccessful                                           70              69
Successful in Spielrichtung                            88              63
Successful gegen Spielrichtung                        347             200
Total Pässe in Spielrichtung                          128             110
Successful in Spielrichtung (Tormitte)                 90              69
Successful gegen Spielrichtung (Tormitte)             345             194
Total in Spielrichtung (Tormitte)                     140             118


In [163]:
passquotetotal1=passsuccessrecdf1.shape[0]/passdf1.shape[0]
passquotetotal2=passsuccessrecdf2.shape[0]/passdf2.shape[0]
passquote901=passsuccessrecdf1['qpass90'].value_counts()[1]/cuid190ges
passquote902=passsuccessrecdf2['qpass90'].value_counts()[1]/cuid290ges
passquote90goal1=passsuccessrecdf1['qpass90goal'].value_counts()[1]/cuid190goalges
passquote90goal2=passsuccessrecdf2['qpass90goal'].value_counts()[1]/cuid290goalges

In [164]:
#Passquote Total: Erfolgreiche Pässe
#Passquote Total in Spielrichtung: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung gespielt wurden
#Passquote Total in Spielrichtung Tormitte: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung Tormitte gespielt wurden
passstatisticproz = {CUID1: [passquotetotal1,passquote901,passquote90goal1],
                 CUID2: [passquotetotal2,passquote902,passquote90goal2]}
passstatisticprozdf = pd.DataFrame(passstatisticproz,columns= [CUID1, CUID2],
                 index=['Passquote Total','Passquote Total in Spielrichtung','Passquote Total in Spielrichtung Tormitte'])
print(passstatisticprozdf)

                                           DFL-CLU-000N9A  DFL-CLU-000N9B
Passquote Total                                  0.854127        0.788856
Passquote Total in Spielrichtung                 0.687500        0.572727
Passquote Total in Spielrichtung Tormitte        0.642857        0.584746


# FCB - TSG

In [165]:
path='C:/Users/Jung/Desktop/Q-Index Master/data/FCB-TSG/'    #arbeit
kpimerged = pd.read_excel('C:/Users/Jung/Desktop/Q-Index Master/data/FCB-TSG/KPI_MERGED.xlsx') #arbeit
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)

xml_positions = etree.parse('C:/Users/Jung/Desktop/Q-Index Master/data/FCB-TSG/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003AVY.xml') ##arbeitsplatop
positions=get_tracking_data(xml_positions)     
positions2=get_tracking_data(xml_positions,"secondHalf")     
##create dataframe from xml input

In [167]:
##get the player ids of the team with club id 1
CUID1 = 'DFL-CLU-00000G'  #Bayern München
#cuid1_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K4_player.xml".format(CUID1)
#cuid1_team_data = DataReader(path+cuid1_player_info)
#cuid1_team_meta_df = cuid1_team_data.create_dataframe()
#cuid1_player_IDs = cuid1_team_meta_df.ObjectId.tolist()
##get the player ids of the team with club id 2
CUID2 = 'DFL-CLU-000002'  #TSG
#cuid2_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K4_player.xml".format(CUID2)
#cuid2_team_data = DataReader(path+cuid2_player_info)
#cuid2_team_meta_df = cuid2_team_data.create_dataframe()
#cuid2_player_IDs = cuid2_team_meta_df.ObjectId.tolist()

In [168]:
matchinfo = {
    CUID1: {
        1:getdirectionofplay(CUID1,1),
        2:getdirectionofplay(CUID1,2),
        "BallPoss":1,
        "TeamID":CUID1,
    },
    CUID2: {
        1:getdirectionofplay(CUID2,1),
        2:getdirectionofplay(CUID2,2),
        "BallPoss":2,
        "TeamID":CUID2,
    },
    1: {
        "Dataframe":positions,
        CUID1:getteamsheet(CUID1,1),
        CUID2:getteamsheet(CUID2,1),
        "Startframe":positions.index.get_level_values(0)[0],
        "Endframe":positions.index.get_level_values(0)[positions.shape[0]-1],
    },
    2: {
        "Dataframe":positions2,
        CUID1:getteamsheet(CUID1,2),
        CUID2:getteamsheet(CUID2,2),
        "Startframe":positions2.index.get_level_values(0)[0],
        "Endframe":positions2.index.get_level_values(0)[positions.shape[0]-1],
    },
    "left to right":{
        "Goalline":52.5
    },
    "right to left":{
        "Goalline":-52.5
    }
}

In [170]:
passdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID1)]
passsuccessdf1=passdf1[(passdf1['EVALUATION']=='successfullyComplete') | (passdf1['EVALUATION']=='successful')]
passsuccessrecdf1=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID1)& (kpimergednew.NORECEIVER==False)]
passunsuccessdf1=passdf1[passdf1['EVALUATION']=='unsuccessful']
passdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['CUID']==CUID2)]
passsuccessdf2=passdf2[(passdf2['EVALUATION']=='successfullyComplete') | (passdf2['EVALUATION']=='successful')]
passsuccessrecdf2=kpimergednew[(kpimergednew['SUBTYPE']=='Pass') & (kpimergednew['EVALUATION']=='successfullyComplete') & (kpimergednew['CUID']==CUID2) & (kpimergednew.NORECEIVER==False)]
passunsuccessdf2=passdf2[passdf2['EVALUATION']=='unsuccessful']

In [171]:
#apply the two functions on the dataframe
# runtime around 3 mins !
passsuccessrecdf1['qpass90']=passsuccessrecdf1.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf2['qpass90']=passsuccessrecdf2.apply(lambda row:qpass90(row),axis=1)
passsuccessrecdf1['qpass90goal']=passsuccessrecdf1.apply(lambda row:qpass90goal(row),axis=1)
passsuccessrecdf2['qpass90goal']=passsuccessrecdf2.apply(lambda row:qpass90goal(row),axis=1)

C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Jung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [172]:
cuid190ges=passsuccessrecdf1['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf1,'successful')
cuid290ges=passsuccessrecdf2['qpass90'].value_counts()[1] + qpass90quote(passunsuccessdf2,'successful')
cuid190goalges=passsuccessrecdf1['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf1,'successful')
cuid290goalges=passsuccessrecdf2['qpass90goal'].value_counts()[1] + qpass90goalquote(passunsuccessdf2,'successful')

In [173]:
#Total: Anzahl aller Pässe
#Successful: erfolgreiche Pässe
#Unsuccessful: fehlgeschlagene Pässe
#Successful in Spielrichtung: Erfolgreiche Pässe in Spielrichtung
#Successful gegen Spielrichtung: Alle Pässe, die nicht in Spielrichtung 90 Grad gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung: Alle Pässe, die in Spielrichtung 90 Grad gespielt wurden 
#Successful in Spielrichtung (Tormitte): Erfolgreiche Pässe in Spielrichtung 90 Grad zur Tormitte
#Successful gegen Spielrichtung (Tormitte): Alle Pässe, die nicht in Spielrichtung 90 Grad zur Tormitte gespielt wurden aber erfolgreich an den Mitpsieler gekommen sind
#Total Pässe in Spielrichtung (Tormitte): Alle Pässe, die in Spielrichtung 90 Grad zur Tormitte gespielt wurden 
passstatistic = {CUID1: [passdf1.shape[0],passsuccessrecdf1.shape[0],passunsuccessdf1.shape[0],passsuccessrecdf1['qpass90'].value_counts()[1],passsuccessrecdf1['qpass90'].value_counts()[0],cuid190ges,passsuccessrecdf1['qpass90goal'].value_counts()[1],passsuccessrecdf1['qpass90goal'].value_counts()[0],cuid190goalges],
                 CUID2: [passdf2.shape[0],passsuccessrecdf2.shape[0],passunsuccessdf2.shape[0],passsuccessrecdf2['qpass90'].value_counts()[1],passsuccessrecdf2['qpass90'].value_counts()[0],cuid290ges,passsuccessrecdf2['qpass90goal'].value_counts()[1],passsuccessrecdf2['qpass90goal'].value_counts()[0],cuid290goalges]}
passstatisticdf = pd.DataFrame(passstatistic,columns= [CUID1, CUID2],
                 index=['Total','Successful','Unsuccessful','Successful in Spielrichtung','Successful gegen Spielrichtung','Total Pässe in Spielrichtung','Successful in Spielrichtung (Tormitte)','Successful gegen Spielrichtung (Tormitte)','Total in Spielrichtung (Tormitte)'])
print(passstatisticdf)

                                           DFL-CLU-00000G  DFL-CLU-000002
Total                                                 611             301
Successful                                            534             233
Unsuccessful                                           76              66
Successful in Spielrichtung                           115              75
Successful gegen Spielrichtung                        409             157
Total Pässe in Spielrichtung                          157             115
Successful in Spielrichtung (Tormitte)                134              72
Successful gegen Spielrichtung (Tormitte)             390             160
Total in Spielrichtung (Tormitte)                     190             119


In [174]:
passquotetotal1=passsuccessrecdf1.shape[0]/passdf1.shape[0]
passquotetotal2=passsuccessrecdf2.shape[0]/passdf2.shape[0]
passquote901=passsuccessrecdf1['qpass90'].value_counts()[1]/cuid190ges
passquote902=passsuccessrecdf2['qpass90'].value_counts()[1]/cuid290ges
passquote90goal1=passsuccessrecdf1['qpass90goal'].value_counts()[1]/cuid190goalges
passquote90goal2=passsuccessrecdf2['qpass90goal'].value_counts()[1]/cuid290goalges

In [175]:
#Passquote Total: Erfolgreiche Pässe
#Passquote Total in Spielrichtung: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung gespielt wurden
#Passquote Total in Spielrichtung Tormitte: Erfolgsquote aller Pässe, die 90 Grad in Spielrichtung Tormitte gespielt wurden
passstatisticproz = {CUID1: [passquotetotal1,passquote901,passquote90goal1],
                 CUID2: [passquotetotal2,passquote902,passquote90goal2]}
passstatisticprozdf = pd.DataFrame(passstatisticproz,columns= [CUID1, CUID2],
                 index=['Passquote Total','Passquote Total in Spielrichtung','Passquote Total in Spielrichtung Tormitte'])
print(passstatisticprozdf)

                                           DFL-CLU-00000G  DFL-CLU-000002
Passquote Total                                  0.873977        0.774086
Passquote Total in Spielrichtung                 0.732484        0.652174
Passquote Total in Spielrichtung Tormitte        0.705263        0.605042
